In [1]:
%%capture
import os
if "COLAB_" in "".join(os.environ.keys()):
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
if 'HF_TOKEN' not in os.environ:
  os.environ['HF_TOKEN'] = input("Enter HF Token:").strip()

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

LLAMA = {
    "src_model": "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "finetuned_model": "saisasanky/Llama-3.1-8B-Instruct-bnb-4bit-aish",
    "gguf_model": "saisasanky/Llama-3.1-8B-Instruct-bnb-4bit-aish-gguf",
    "chat_template": "llama-3.1",
    "instruction_template": "<|start_header_id|>user<|end_header_id|>\n\n",
    "response_template": "<|start_header_id|>assistant<|end_header_id|>\n\n",
}
QWEN = {
    "src_model": "unsloth/Qwen2.5-Coder-14B-Instruct-bnb-4bit",
    "finetuned_model": "saisasanky/Qwen2.5-Coder-14B-Instruct-bnb-4bit-aish",
    "gguf_model":  "saisasanky/Qwen2.5-Coder-14B-Instruct-bnb-4bit-aish-gguf",
    "chat_template": "qwen-2.5",
    "instruction_template": "<|im_start|>user\n",
    "response_template": "<|im_start|>assistant\n",
}

model_config = QWEN

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_config["src_model"],
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.7 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = model_config["chat_template"],
)

from datasets import load_dataset
dataset = load_dataset("westenfelder/NL2SH-ALFA", "train", split = "train").shuffle()


In [5]:
# prompt: user apply_chat_template on dataset where each row is similar to
# {'nl': 'show the free space on all filesystems', 'bash': 'df -h'}

def formatting_prompts_func(examples):
    nls = examples['nl']
    bashs = examples['bash']

    texts = [tokenizer.apply_chat_template(
        [{"role": "system", "content": "You are an assistant that provides exact bash command for given input"},
         {"role": "user", "content": nl},
         {"role": "assistant", "content": bash}],
        tokenize=False, add_generation_prompt=False)
            for nl,bash in zip(nls,bashs)]
    return {'text': texts}

dataset = dataset.map(formatting_prompts_func, batched=True)
dataset[0]['text']

Map:   0%|          | 0/40639 [00:00<?, ? examples/s]

'<|im_start|>system\nYou are an assistant that provides exact bash command for given input<|im_end|>\n<|im_start|>user\nRun query on database file and keep the interactive shell open<|im_end|>\n<|im_start|>assistant\nduckdb path/to/dbfile -cmd "SELECT DISTINCT * FROM tbl"<|im_end|>\n'

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [6]:
dataset[30]["text"]

'<|im_start|>system\nYou are an assistant that provides exact bash command for given input<|im_end|>\n<|im_start|>user\nList all running units, ordered by the time they took to initialize<|im_end|>\n<|im_start|>assistant\nsystemd-analyze blame<|im_end|>\n'

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 32,
        warmup_steps = 10,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/40639 [00:00<?, ? examples/s]

In [8]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
9.857 GB of memory reserved.


In [9]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = model_config["instruction_template"],
    response_part = model_config["response_template"],
)

Map (num_proc=2):   0%|          | 0/40639 [00:00<?, ? examples/s]

In [10]:
tokenizer.decode(trainer.train_dataset[5]['input_ids'])

'<|im_start|>system\nYou are an assistant that provides exact bash command for given input<|im_end|>\n<|im_start|>user\nFind all files with the .png extension in the Desktop directory and delete them.<|im_end|>\n<|im_start|>assistant\nfind ~/Desktop -name "*.png" -exec rm {} \\;<|im_end|>\n'

In [11]:
space = tokenizer(" ",add_special_tokens=False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]['labels']])

'                                         find ~/Desktop -name "*.png" -exec rm {} \\;<|im_end|>\n'

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 40,639 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 32
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 32 x 1) = 64
 "-____-"     Trainable parameters = 137,625,600/14,000,000,000 (0.98% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: sasank to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.707400
2,1.662000
3,1.633600
4,1.941900
5,1.944900
6,2.029800
7,1.859800
8,1.980200
9,1.439700
10,1.640700


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = model_config["chat_template"],
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user","content": "Command to List all the contents in the current folder"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors = "pt",
).to("cuda")
outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True, skip_special_tokens = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCommand to List all the contents in the current folder<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nls<|eot_id|>']

In [ ]:
model.push_to_hub(model_config["finetuned_model"], token = os.environ['HF_TOKEN'])
tokenizer.push_to_hub(model_config["finetuned_model"], token = os.environ['HF_TOKEN'])

In [ ]:
model.push_to_hub_gguf(
    model_config["gguf_repo"],
    tokenizer,
    token = os.environ['HF_TOKEN'],
    quantization_method = 'q8_0'
)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.13 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 53%|█████▎    | 17/32 [00:02<00:01, 12.75it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:05<00:00,  2.03s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving saisasanky/Llama-3.1-8B-Instruct-4bit-aish_gguf/pytorch_model-00001-of-00004.bin...
Unsloth: Saving saisasanky/Llama-3.1-8B-Instruct-4bit-aish_gguf/pytorch_model-00002-of-00004.bin...
Unsloth: Saving saisasanky/Llama-3.1-8B-Instruct-4bit-aish_gguf/pytorch_model-00003-of-00004.bin...
Unsloth: Saving saisasanky/Llama-3.1-8B-Instruct-4bit-aish_gguf/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at saisasanky/Llama-3.1-8B-Instruct-4bit-aish_gguf into q8_0 GGUF format.
The output location will be /content/saisasanky/Llama-3.1-8B-Instruct-4bit-aish_gguf/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Llama-3.1-8B-Instruct-4bit-aish_gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/saisasanky/Llama-3.1-8B-Instruct-4bit-aish_gguf
